In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

In [26]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [5]:
dataset="D:\dataset\final\cleandata"

In [6]:
Corpus = pd.read_csv(r"D:\dataset\final\cleandata.csv",encoding='latin-1')
Corpus['title'].dropna(inplace=True)

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['title'],Corpus['label'],test_size=0.3)

In [30]:

print(len(Train_X))
print(len(Train_Y))
print(len(Test_X))
print(len(Test_Y))

1244422
1244422
533325
533325


In [31]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y.astype(str))
Test_Y = Encoder.fit_transform(Test_Y.astype(str))

In [32]:
training_labels_final = np.array(Train_Y)
testing_labels_final = np.array(Test_Y)

In [8]:
vocab_size = 120000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(Train_X)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(Train_X)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(Test_X)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [35]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 16)           1920000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 7         
Total params: 1,931,533
Trainable params: 1,931,533
Non-trainable params: 0
_________________________________________________________________


In [36]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 1244422 samples, validate on 533325 samples
Epoch 1/10
1244422/1244422 [==============================] - 1101s 885us/sample - loss: 0.0111 - acc: 0.9975 - val_loss: 0.0036 - val_acc: 0.9989
Epoch 2/10
1244422/1244422 [==============================] - 1169s 940us/sample - loss: 0.0024 - acc: 0.9992 - val_loss: 0.0036 - val_acc: 0.9987
Epoch 3/10
1244422/1244422 [==============================] - 1127s 906us/sample - loss: 0.0019 - acc: 0.9994 - val_loss: 0.0039 - val_acc: 0.9988
Epoch 4/10
1244422/1244422 [==============================] - 1186s 953us/sample - loss: 0.0016 - acc: 0.9995 - val_loss: 0.0040 - val_acc: 0.9988
Epoch 5/10
1244422/1244422 [==============================] - 1151s 925us/sample - loss: 0.0014 - acc: 0.9996 - val_loss: 0.0041 - val_acc: 0.9982
Epoch 6/10
1244422/1244422 [==============================] - 1020s 820us/sample - loss: 0.0012 - acc: 0.9996 - val_loss: 0.0044 - val_acc: 0.9981
Epoch 7/10
1244422/1244422 [==============================] - 110

In [42]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_sentence(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_sentence(padded[0])+"\n")
print(Train_X[2])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? my uncle is angry

riding hungarian tittyfuck cumshots striptease naturaltits blowjob bubblebutt hardcore deepthroat bigtits brunette shaved



In [54]:
#embedding projector 

import io
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    try:
        word = reverse_word_index[word_num]
        embeddings = weights[word_num]
        out_m.write(word + "\n")
        out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
    except KeyError:
        pass
out_v.close()
out_m.close()

In [11]:
padding_type='post'
sentence = ["let's go "]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
x=new_model.predict(padded)
if x>0.5:
    print("inapprpriate context")
else:
    print("appropriate context")

inapprpriate context


In [66]:
#save model 
model.save('LSTMmodel')



In [3]:
#load model 
new_model = tf.keras.models.load_model('LSTMmodel')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
#embedding using a trainable model 
# Note this is the 100 dimension version of GloVe from Stanford
# I unzipped and hosted it on my site to make this notebook easier
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt
embeddings_index = {};
with open('/tmp/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, 100, input_length=max_length, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [7]:
vectorizer = CountVectorizer()
vectorizer.fit(Corpus['title'])


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [8]:
vocab_size=len(vectorizer.vocabulary_)

In [9]:
print (vocab_size)

129818


In [15]:
sample_string = 'Hello TensorFlow.'

encoded_string = vectorizer.transform([sample_string])
encoded_string=encoded_string.toarray()
print('Encoded string is {}'.format(encoded_string))

Encoded string is [[0 0 0 ... 0 0 0]]


In [10]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [17]:
Train_X = Train_X.shuffle(BUFFER_SIZE)
Train_X = Train_X.padded_batch(BATCH_SIZE)

Test_X = Test_X.padded_batch(BATCH_SIZE)

AttributeError: 'Series' object has no attribute 'shuffle'

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [19]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [20]:
history = model.fit(Train_X,Train_Y,
                    epochs=10,
                    validation_data=(Test_X,Test_Y), 
                    validation_steps=30)

Train on 1244422 samples, validate on 533325 samples
Epoch 1/10


ValueError: could not convert string to float: 'You told me Tom was handsome and he really is\n'